In [1]:
%matplotlib ipympl
%load_ext line_profiler
%load_ext Cython
%load_ext autoreload
%autoreload 2
#import gym
#import gym_watten
#from gym_watten.envs.watten_env import WattenEnv
import numpy as np
from time import sleep
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import pydot_ng as pydot
from io import BytesIO
import pickle
from src.Storage import Storage
from src.MCTS import MCTS
from src.LookUp import LookUp
from src.KerasModel import KerasModel
from src.TinyDnnModel import TinyDnnModel
from src.Game import Game
from src.ModelRating import ModelRating
from gym_watten.envs.watten_env import WattenEnv
import sys
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
env = WattenEnv()
model = LookUp()
best_model = LookUp()
storage = Storage()
mcts = MCTS()
rating = ModelRating(env)
game = Game(env)

In [2]:
MEMORY_SIZE = 300000
SAMPLES_PER_STATE = 1

BATCH_SIZE = 256
TRAINING_LOOPS = 100
EXPLORATION = 1

EPSILON = 0.2
ALPHA = 0.8

In [3]:
%%cython_mainpxd

from libcpp cimport bool
from libcpp.map cimport map
from libcpp.string cimport string
from libcpp.vector cimport vector
from gym_watten.envs.watten_env cimport Observation, WattenEnv, State


cdef enum:
    EPISODES = 75
    MCTS_SIMS = 20
    OBJECTIVE_OPPONENT = False



cdef struct MCTSState:
    vector[MCTSState] childs
    int n
    float w
    float v
    float p
    State env_state
    float end_v 
    int current_player
    bool is_root
    

cdef class MCTSInternal:
    @staticmethod
    cdef mcts_sample(WattenEnv env, MCTSState* state, LookUp model)
    @staticmethod
    cdef object calc_q(MCTSState* state, int player)